Simple pipeline and plyground for analizing temporal graphs by using DMGI.

Currently on mock data

In [ ]:
import sys
import logging
import torch
sys.path.append("../")
from temporal_graphs.src.data.mock_data import get_temporal_mock_graph
from temporal_graphs.src.data import make_dataset
from temporal_graphs.src.temporal_graph.build_graph import create_torch_temporal_graph_from_df
from temporal_graphs.src.models.dmgi_trainer import DMGITrainer

logging.basicConfig(level="INFO")
prefix = "dmgi"

# Data

In [ ]:
number_of_nodes_frome_each = 300
graph = make_dataset.get_subreddit_links(prefix="../")
nodes = set(set(graph["SOURCE_SUBREDDIT"].value_counts()[:number_of_nodes_frome_each].index))
nodes.update(set(graph["TARGET_SUBREDDIT"].value_counts()[:number_of_nodes_frome_each].index))
graph = graph[(graph["SOURCE_SUBREDDIT"].isin(nodes)) & (graph["TARGET_SUBREDDIT"].isin(nodes))]

graph = make_dataset.add_date_group(graph, "TIMESTAMP", number_of_windows=20, window=12, offset=3)
graph = graph[['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT', 'date_group']]
graph.columns = ["node_1", "node_2", "date_group"]
nodes = set(graph["node_1"].unique())
nodes.update(set(graph["node_2"].unique()))

node_name_dict = {i: name for i, name in enumerate(nodes)}
name_node_dict = {v: k for k, v in node_name_dict.items()}

graph["node_1"] = graph["node_1"].apply(lambda x: name_node_dict[x])
graph["node_2"] = graph["node_2"].apply(lambda x: name_node_dict[x])

graph = create_torch_temporal_graph_from_df(graph, save=False, path_prefix="../", feature_size=256)

In [ ]:
graph

# Train

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
device = torch.device("cpu")
device

In [ ]:
model = DMGITrainer(data=graph, out_channels=128, conv_name="GCNConv", normalize_features=False, device=device, alpha=0.0001)

In [ ]:
model.train(epochs=5000, learning_rate=0.0001, 
            weight_decay=0.00005, 
            print_every_n_epoch=100)

In [ ]:
model.save(path=f"../temporal_graphs/models/{prefix}_model.pt")

In [ ]:
embeddings = model.get_embeddings()

In [ ]:
torch.save(embeddings, f"../temporal_graphs/models/{prefix}_embeddings.pt")

In [ ]:
embeddings = embeddings.cpu().detach().numpy().astype("double")